In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import requests
from pandas.io.json import json_normalize

In [2]:
intake = requests.get("https://data.austintexas.gov/resource/fdzn-9yqv.json").json()


In [3]:
intake_df = json_normalize(data=intake).set_index("animal_id")

In [4]:
outcomes = requests.get("https://data.austintexas.gov/resource/9t4d-g238.json").json()

In [5]:
outcomes_df = json_normalize(data=outcomes).set_index("animal_id")

In [6]:
pd.concat([intake_df, outcomes_df], axis=0)

C:\Users\kenak\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,age_upon_intake,age_upon_outcome,animal_type,breed,color,date_of_birth,datetime,datetime2,found_location,intake_condition,intake_type,monthyear,name,outcome_subtype,outcome_type,sex_upon_intake,sex_upon_outcome
animal_id,,,,,,,,,,,,,,,,,
A786884,2 years,NaN,Dog,Beagle Mix,Tricolor,NaN,2019-01-03T16:19:00.000,2019-01-03T16:19:00.000,2501 Magin Meadow Dr in Austin (TX),Normal,Stray,NaN,*Brock,NaN,NaN,Neutered Male,NaN
A706918,8 years,NaN,Dog,English Springer Spaniel,White/Liver,NaN,2015-07-05T12:59:00.000,2015-07-05T12:59:00.000,9409 Bluegrass Dr in Austin (TX),Normal,Stray,NaN,Belle,NaN,NaN,Spayed Female,NaN
A724273,11 months,NaN,Dog,Basenji Mix,Sable/White,NaN,2016-04-14T18:43:00.000,2016-04-14T18:43:00.000,2818 Palomino Trail in Austin (TX),Normal,Stray,NaN,Runster,NaN,NaN,Intact Male,NaN
A778404,4 years,NaN,Dog,German Shepherd Mix,Black/Tan,NaN,2018-08-14T10:19:00.000,2018-08-14T10:19:00.000,Austin (TX),Normal,Owner Surrender,NaN,Max,NaN,NaN,Intact Male,NaN
A665644,4 weeks,NaN,Cat,Domestic Shorthair Mix,Calico,NaN,2013-10-21T07:59:00.000,2013-10-21T07:59:00.000,Austin (TX),Sick,Stray,NaN,NaN,NaN,NaN,Intact Female,NaN
A682524,4 years,NaN,Dog,Doberman Pinsch/Australian Cattle Dog,Tan/Gray,NaN,2014-06-29T10:38:00.000,2014-06-29T10:38:00.000,800 Grove Blvd in Austin (TX),Normal,Stray,NaN,Rio,NaN,NaN,Neutered Male,NaN
A743852,2 years,NaN,Dog,Labrador Retriever Mix,Chocolate,NaN,2017-02-18T12:46:00.000,2017-02-18T12:46:00.000,Austin (TX),Normal,Owner Surrender,NaN,Odin,NaN,NaN,Neutered Male,NaN
A708452,2 years,NaN,Dog,Labrador Retriever Mix,Black/White,NaN,2015-07-30T14:37:00.000,2015-07-30T14:37:00.000,Austin (TX),Normal,Public Assist,NaN,Mumble,NaN,NaN,Intact Male,NaN
A774147,4 weeks,NaN,Cat,Domestic Shorthair Mix,Black/White,NaN,2018-06-11T07:45:00.000,2018-06-11T07:45:00.000,6600 Elm Creek in Austin (TX),Injured,Stray,NaN,NaN,NaN,NaN,Intact Female,NaN


In [7]:
intake_df

,age_upon_intake,animal_type,breed,color,datetime,datetime2,found_location,intake_condition,intake_type,name,sex_upon_intake
animal_id,,,,,,,,,,,
A786884,2 years,Dog,Beagle Mix,Tricolor,2019-01-03T16:19:00.000,2019-01-03T16:19:00.000,2501 Magin Meadow Dr in Austin (TX),Normal,Stray,*Brock,Neutered Male
A706918,8 years,Dog,English Springer Spaniel,White/Liver,2015-07-05T12:59:00.000,2015-07-05T12:59:00.000,9409 Bluegrass Dr in Austin (TX),Normal,Stray,Belle,Spayed Female
A724273,11 months,Dog,Basenji Mix,Sable/White,2016-04-14T18:43:00.000,2016-04-14T18:43:00.000,2818 Palomino Trail in Austin (TX),Normal,Stray,Runster,Intact Male
A778404,4 years,Dog,German Shepherd Mix,Black/Tan,2018-08-14T10:19:00.000,2018-08-14T10:19:00.000,Austin (TX),Normal,Owner Surrender,Max,Intact Male
A665644,4 weeks,Cat,Domestic Shorthair Mix,Calico,2013-10-21T07:59:00.000,2013-10-21T07:59:00.000,Austin (TX),Sick,Stray,NaN,Intact Female
A682524,4 years,Dog,Doberman Pinsch/Australian Cattle Dog,Tan/Gray,2014-06-29T10:38:00.000,2014-06-29T10:38:00.000,800 Grove Blvd in Austin (TX),Normal,Stray,Rio,Neutered Male
A743852,2 years,Dog,Labrador Retriever Mix,Chocolate,2017-02-18T12:46:00.000,2017-02-18T12:46:00.000,Austin (TX),Normal,Owner Surrender,Odin,Neutered Male
A708452,2 years,Dog,Labrador Retriever Mix,Black/White,2015-07-30T14:37:00.000,2015-07-30T14:37:00.000,Austin (TX),Normal,Public Assist,Mumble,Intact Male
A774147,4 weeks,Cat,Domestic Shorthair Mix,Black/White,2018-06-11T07:45:00.000,2018-06-11T07:45:00.000,6600 Elm Creek in Austin (TX),Injured,Stray,NaN,Intact Female


In [9]:
outcomes_df

,age_upon_outcome,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome
animal_id,,,,,,,,,,,
A788476,1 year,Dog,Pit Bull Mix,Brown,2017-08-03T00:00:00.000,2019-03-08T14:53:00.000,2019-03-08T14:53:00.000,*Leia,NaN,Adoption,Spayed Female
A789646,1 year,Dog,Bull Terrier Mix,Tan/White,2018-02-25T00:00:00.000,2019-03-08T14:35:00.000,2019-03-08T14:35:00.000,P!Nkie,NaN,Adoption,Spayed Female
A789758,3 years,Dog,Boxer/Cardigan Welsh Corgi,White/Black,2016-03-02T00:00:00.000,2019-03-08T14:31:00.000,2019-03-08T14:31:00.000,Dyna,NaN,Return to Owner,Intact Female
A789940,2 months,Cat,Domestic Shorthair Mix,Tortie,2019-01-02T00:00:00.000,2019-03-08T14:23:00.000,2019-03-08T14:23:00.000,NaN,NaN,Adoption,Spayed Female
A790029,10 months,Cat,Domestic Shorthair Mix,Blue/White,2018-05-03T00:00:00.000,2019-03-08T14:07:00.000,2019-03-08T14:07:00.000,Flora,NaN,Return to Owner,Spayed Female
A789353,10 years,Cat,Domestic Shorthair Mix,Blue,2009-02-20T00:00:00.000,2019-03-08T14:07:00.000,2019-03-08T14:07:00.000,NaN,Partner,Transfer,Spayed Female
A789895,2 years,Cat,Domestic Shorthair Mix,Brown Tabby,2017-03-01T00:00:00.000,2019-03-08T14:06:00.000,2019-03-08T14:06:00.000,*Mercy,Partner,Transfer,Neutered Male
A789929,4 years,Dog,Miniature Schnauzer Mix,White,2015-03-07T00:00:00.000,2019-03-08T13:50:00.000,2019-03-08T13:50:00.000,NaN,Partner,Transfer,Neutered Male
A789686,5 years,Dog,Pit Bull/Australian Cattle Dog,White/Red,2014-02-25T00:00:00.000,2019-03-08T12:50:00.000,2019-03-08T12:50:00.000,Shorty,NaN,Return to Owner,Neutered Male


In [13]:
outcomes_df.sex_upon_outcome.value_counts()

Neutered Male    361
Spayed Female    312
Intact Male      133
Intact Female    108
Unknown           86
Name: sex_upon_outcome, dtype: int64

In [11]:
intake_df.animal_type.value_counts()

Dog      575
Cat      369
Other     53
Bird       3
Name: animal_type, dtype: int64